In [39]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
import sys

sys.path.append("../")

In [41]:
import fof.dataloader
import transformers as tr
from pathlib import Path

In [50]:
loader = fof.dataloader.ScicapDataModule(
  "First-Sentence",
  tokenizer=tr.AutoTokenizer.from_pretrained("distilbert-base-uncased"),
  root=Path("../scicap_data"),
  batch_size=4,
  caption_type="normalized",
  num_workers=0)
train = loader.train_dataloader()

Initializing SCICAP training dataset
Initializing SCICAP testing dataset
Initializing SCICAP validation dataset
Time taken: 0.06000995635986328


In [52]:
batch = next(iter(train))
len(batch)

6

In [53]:
for key in ["references", "labels", "title", "id"]:
    print(key, batch[key])

references ['ting\nthe data in between the execution of the prefetch and the subsequent access in the last round.\nFigure 9 is complementary to Figure 8. It represents the\nnumber of guesses that an attacker would require to', '', ' procedure)\nof the RC classifiers varies on the dataset in Tab. I.\nWe report the average results in Fig. 7.\n\nClassification accuracy\n\n88\n0.32\n\n87\n86\n\n0.3\n\n85\n84\n25\n\n50\n\n75\n\n100\n\n125\n\n0.28\n150\n\nExecution tim', "every day diet. As a result, the set of distinct ingredients\nroughly follows Heap's law, as seen in Fig. 4, with an exponent around 0.64. According to the method in\n1\npaper [20], the exponent of Zipf's law "]
labels ['key candidate search space for the NUM bits of the key of the s-box aes implementation of openssl retrieved using the tsx-based detection and method NUM .', 'throughput graph .', 'average classification accuracy and execution time for different dimensions d of the space with reduced dimensionality .', 'the 